<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 10


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

Создать базовый класс Invoice в C#, который будет представлять информацию о
фактурах за поставленные товары или оказанные услуги. На основе этого класса
разработать 2-3 производных класса, демонстрирующих принципы наследования и
полиморфизма. В каждом из классов должны быть реализованы новые атрибуты и
методы, а также переопределены некоторые методы базового класса для
демонстрации полиморфизма.
Требования к базовому классу Invoice:
• Атрибуты: Номер фактуры (InvoiceNumber), Дата выдачи (IssueDate), Общая
сумма (TotalAmount).
• Методы:
o
o CalculateTotal(): метод для расчета общей суммы по фактуре.
o AddLine(LineItem lineItem): метод для добавления позиции в фактуру.
o RemoveLine(LineItem lineItem): метод для удаления позиции из
фактуры.
Требования к производным классам:
1. ТоварнаяФактура (GoodsInvoice): Должна содержать дополнительные
атрибуты, такие как Дата поставки (SupplyDate). Метод AddLine() должен
быть переопределен для добавления информации о дате поставки товара
при добавлении позиции.
2. УслуговаяФактура (ServiceInvoice): Должна содержать дополнительные
атрибуты, такие как Дата оказания услуги (ServiceDate).
Метод RemoveLine() должен быть переопределен для добавления
информации о причине аннулирования услуги при удалении позиции.
3. КомбинированнаяФактура (CombinedInvoice) (если требуется третий класс):
Должна содержать дополнительные атрибуты, такие как Наличие возврата
(ReturnAllowed). Метод CalculateTotal() должен быть переопределен для
учета возможного возврата товара или услуги при расчете общей суммы.

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) исользуйтие в проекте коллекции, делегаты, события.


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;

// Делегат для уведомления
public delegate void InvoiceUpdatedHandler(object sender, EventArgs e);

// Интерфейс для счета дополнительных деталей
public interface IInvoiceDetails
{
    void DisplayInvoiceSummary();
}

// Интерфейс для счета
public interface IInvoice : IInvoiceDetails
{
    void CalculateTotal();
    void AddLine(LineItem lineItem);
    void RemoveLine(string description);
    void MarkAsPaid();
    void DisplayInvoiceDetails();
    void DisplayInvoiceDetails(bool includeLineItems);
}

// Абстрактный класс счета
public abstract class Invoice : IInvoice
{
    // Дополнительные атрибуты
    public string CreatedByDepartment { get; set; } // Отдел, создавший счет
    public decimal Tax { get; set; } // Налог на счет
    public string PaymentTerms { get; set; } // Условия оплаты
    public string Notes { get; set; } // Дополнительные заметки

    public string _invoiceNumber;
    public DateTime _issueDate;
    public decimal _totalAmount;
    protected List<LineItem> LineItems { get; set; }
    public string _currency;
    public string _customerName;
    public bool IsPaid { get; private set; }
    public string InvoiceDescription { get; set; }
    public string CreatedBy { get; set; }
    public DateTime DueDate { get; set; }
    
    // Новые атрибуты
    public string Status { get; set; } // Статус обработки счета
    public bool IsArchived { get; set; } // Статус архивирования

    // Событие, которое будет вызываться при обновлении счета
    public event InvoiceUpdatedHandler InvoiceUpdated;

    public Invoice(string _invoiceNumber, DateTime _issueDate, string _customerName, string invoiceDescription, string createdBy, DateTime dueDate, string department)
    {
        InvoiceNumber = _invoiceNumber;
        IssueDate = _issueDate;
        LineItems = new List<LineItem>();
        _currency = "USD";
        CustomerName = _customerName;
        IsPaid = false;
        InvoiceDescription = invoiceDescription;
        CreatedBy = createdBy;
        DueDate = dueDate;
        CreatedByDepartment = department;
        Status = "Создан"; // Изначально статус создан
        IsArchived = false; // Изначально не архивирован
    }

    public decimal TotalAmount
    {
        get { return _totalAmount; }
        set
        {
            if (value >= 0)
                _totalAmount = value;
            else
                throw new ArgumentOutOfRangeException("Общая сумма не может быть отрицательной");
        }
    }

    public string InvoiceNumber
    {
        get { return _invoiceNumber; }
        set { _invoiceNumber = value; }
    }

    public DateTime IssueDate
    {
        get { return _issueDate; }
        set { _issueDate = value; }
    }

    public string Currency
    {
        get { return _currency; }
    }

    public string CustomerName
    {
        get { return _customerName; }
        set { _customerName = value; }
    }

    public virtual void CalculateTotal()
    {
        TotalAmount = 0;
        foreach (var item in LineItems)
        {
            TotalAmount += item.Amount;
        }
        TotalAmount += Tax; // Добавить налог к общей сумме
        OnInvoiceUpdated(EventArgs.Empty); // Вызываем событие при обновлении суммы
    }

    public virtual void UpdateTax(decimal tax)
    {
        Tax = tax; // Установка налога
        CalculateTotal(); // Пересчет общей суммы
    }

    public virtual void AddLine(LineItem lineItem)
    {
        LineItems.Add(lineItem);
        CalculateTotal();
        OnInvoiceUpdated(EventArgs.Empty); // Сообщаем об обновлении
    }

    public virtual void RemoveLine(string description)
    {
        var lineItem = LineItems.FirstOrDefault(item => item.Description == description);
        if (lineItem != null)
        {
            LineItems.Remove(lineItem);
            CalculateTotal();
            OnInvoiceUpdated(EventArgs.Empty); // Сообщаем об обновлении
        }
    }

    public virtual void MarkAsPaid()
    {
        IsPaid = true;
        Status = "Оплачен"; // Обновляем статус
        Console.WriteLine($"Счет {InvoiceNumber} отмечен как оплаченный.");
        OnInvoiceUpdated(EventArgs.Empty); // Сообщаем об обновлении
    }

    public virtual void DisplayInvoiceDetails()
    {
        Console.WriteLine($"Счет: {InvoiceNumber}, Дата: {IssueDate.ToShortDateString()}, Клиент: {CustomerName}, Валюта: {Currency}, Оплачен: {IsPaid}, Описание: {InvoiceDescription}, Создано: {CreatedBy}, Срок платежа: {DueDate.ToShortDateString()}, Отдел: {CreatedByDepartment}, Налог: {Tax}, Условия оплаты: {PaymentTerms}, Примечания: {Notes}, Статус: {Status}, Архивирован: {IsArchived}");
    }

    public virtual void DisplayInvoiceDetails(bool includeLineItems)
    {
        DisplayInvoiceDetails();
        if (includeLineItems)
        {
            Console.WriteLine("Строки:");
            foreach (var item in LineItems)
            {
                Console.WriteLine($"- {item.Description}: {item.Amount}");
            }
        }
    }

    public void DisplayInvoiceSummary()
    {
        Console.WriteLine($"Счет {InvoiceNumber}: Общая сумма - {TotalAmount}, Статус - {(IsPaid ? "Оплачен" : "Не оплачен")}, Обработан: {Status}, Архивирован: {IsArchived}");
    }

    // Метод, который будет вызываться для уведомления об обновлении
    protected virtual void OnInvoiceUpdated(EventArgs e)
    {
        InvoiceUpdated?.Invoke(this, e);
    }
}

// Упрощенный класс строки
public class LineItem
{
    public string _description;
    public decimal _amount;

    public string Description
    {
        get { return _description; }
        set { _description = value; }
    }

    public decimal Amount
    {
        get { return _amount; }
        set
        {
            if (value >= 0)
                _amount = value;
            else
                throw new ArgumentOutOfRangeException("Стоимость не может быть отрицательной");
        }
    }

    public LineItem(string _description, decimal _amount)
    {
        Description = _description;
        Amount = _amount;
    }
}

// Класс товара для счетов
public class GoodsInvoice : Invoice
{
    public DateTime _supplyDate;
    public string _supplierName;
    public string _deliveryMethod;
    public string WarehouseLocation { get; set; } // Дополнительный атрибут
    public DateTime ExpectedDeliveryDate { get; set; } // Ожидаемая дата доставки

    public GoodsInvoice(string _invoiceNumber, DateTime _issueDate, DateTime _supplyDate, string currency, string customerName, string supplierName, string deliveryMethod, string invoiceDescription, string createdBy, DateTime dueDate, string department) 
        : base(_invoiceNumber, _issueDate, customerName, invoiceDescription, createdBy, dueDate, department)
    {
        SupplyDate = _supplyDate;
        SupplierName = supplierName;
        DeliveryMethod = deliveryMethod;
    }

    public DateTime SupplyDate
    {
        get { return _supplyDate; }
        set { _supplyDate = value; }
    }

    public string SupplierName
    {
        get { return _supplierName; }
        set { _supplierName = value; }
    }

    public string DeliveryMethod
    {
        get { return _deliveryMethod; }
        set { _deliveryMethod = value; }
    }

    public void DisplaySupplierInfo()
    {
        Console.WriteLine($"Поставщик: {SupplierName}, Способ доставки: {DeliveryMethod}, Местонахождение склада: {WarehouseLocation}, Ожидаемая дата доставки: {ExpectedDeliveryDate.ToShortDateString()}");
    }

    public override void DisplayInvoiceDetails(bool includeLineItems = false)
    {
        base.DisplayInvoiceDetails(includeLineItems);
        DisplaySupplierInfo();
    }
}

// Класс для услуг
public class ServiceInvoice : Invoice
{
    public DateTime _serviceDate;
    public string _serviceProvider;
    public string _serviceDescription;
    public string ServiceLocation { get; set; } // Местонахождение обслуживания
    public int EstimatedHours { get; set; } // Оценка часов

    public ServiceInvoice(string _invoiceNumber, DateTime _issueDate, DateTime _serviceDate, string currency, string customerName, string serviceProvider, string serviceDescription, string invoiceDescription, string createdBy, DateTime dueDate, string department) 
        : base(_invoiceNumber, _issueDate, customerName, invoiceDescription, createdBy, dueDate, department)
    {
        ServiceDate = _serviceDate;
        ServiceProvider = serviceProvider;
        ServiceDescription = serviceDescription;
    }

    public DateTime ServiceDate
    {
        get { return _serviceDate; }
        set { _serviceDate = value; }
    }

    public string ServiceProvider
    {
        get { return _serviceProvider; }
        set { _serviceProvider = value; }
    }

    public string ServiceDescription
    {
        get { return _serviceDescription; }
        set { _serviceDescription = value; }
    }

    public void DisplayServiceInfo()
    {
        Console.WriteLine($"Поставщик услуги: {ServiceProvider}, Описание услуги: {ServiceDescription}, Местонахождение услуги: {ServiceLocation}, Ожидаемое время: {EstimatedHours} часов");
    }

    public override void DisplayInvoiceDetails(bool includeLineItems = false)
    {
        base.DisplayInvoiceDetails(includeLineItems);
        DisplayServiceInfo();
    }
}

// Класс комбинированного счета
public class CombinedInvoice : Invoice
{
    public bool ReturnAllowed { get; private set; }
    public string _invoiceType;
    public decimal Discount { get; set; }
    public string ApplicableLaw { get; set; } // Применимое законодательство
    public bool IsFinalInvoice { get; set; } // Финальный счет
    public string PaymentMethod { get; set; } // Методы оплаты

    public string InvoiceType
    {
        get { return _invoiceType; }
        set { _invoiceType = value; }
    }

    public CombinedInvoice(string invoiceNumber, DateTime issueDate, bool returnAllowed, string customerName, string invoiceDescription, string createdBy, DateTime dueDate, string department) 
        : base(invoiceNumber, issueDate, customerName, invoiceDescription, createdBy, dueDate, department)
    {
        ReturnAllowed = returnAllowed;
        InvoiceType = "Комбинированный";
        Discount = 0;
    }

    public void ApplyDiscount(decimal discountPercentage)
    {
        if (discountPercentage < 0 || discountPercentage > 100)
            throw new ArgumentOutOfRangeException("Скидка должна быть от 0 до 100%.");

        decimal discountAmount = (TotalAmount * discountPercentage) / 100;
        Discount += discountAmount;
        TotalAmount -= discountAmount;
        Console.WriteLine($"Скидка в размере {discountAmount} применена.");
    }

    public decimal CalculateFinalAmount()
    {
        return TotalAmount;
    }

    public void DisplayCombinedInvoiceInfo()
    {
        Console.WriteLine($"Тип счета: {InvoiceType}, Скидка: {Discount}, Итоговая сумма: {CalculateFinalAmount()}, Применимое законодательство: {ApplicableLaw}, " + 
                          $"Финальный счет: {(IsFinalInvoice ? "Да" : "Нет")}, Методы оплаты: {PaymentMethod}");
    }

    public override void RemoveLine(string description)
    {
        if (ReturnAllowed)
        {
            base.RemoveLine(description);
            Console.WriteLine($"Товар {description} возвращен, общая сумма пересчитана.");
        }
        else
        {
            Console.WriteLine($"Товар {description} не может быть возвращен.");
        }
    }

    public override void DisplayInvoiceDetails(bool includeLineItems = false)
    {
        base.DisplayInvoiceDetails(includeLineItems);
        DisplayCombinedInvoiceInfo();
    }
}

// Пример использования
class Program
{
    static void Main()
    {
        LineItem item1 = new LineItem("Товар A", 100);
        LineItem item2 = new LineItem("Товар B", 200);
        
        GoodsInvoice goodsInvoice = new GoodsInvoice("GI001", DateTime.Now, DateTime.Now.AddDays(10), "USD", "Клиент 1", "Поставщик A", "Курьерская доставка", "Покупка товаров", "Пользователь1", DateTime.Now.AddDays(30), "Отдел продаж");
        goodsInvoice.InvoiceUpdated += InvoiceUpdatedHandler; // Подписка на событие
        goodsInvoice.AddLine(item1);
        goodsInvoice.AddLine(item2);
        goodsInvoice.UpdateTax(15); // Добавляем налог
        goodsInvoice.DisplayInvoiceDetails(true);
        goodsInvoice.WarehouseLocation = "Склад 1";
        goodsInvoice.ExpectedDeliveryDate = DateTime.Now.AddDays(5);
        goodsInvoice.DisplayInvoiceSummary();
        Console.WriteLine();

        LineItem serviceItem1 = new LineItem("Услуга A", 300);
        
        ServiceInvoice serviceInvoice = new ServiceInvoice("SI001", DateTime.Now, DateTime.Now.AddDays(5), "USD", "Клиент 2", "Поставщик услуги B", "Ремонт оборудования", "Покупка услуг", "Пользователь2", DateTime.Now.AddDays(20), "Отдел поддержки");
        serviceInvoice.InvoiceUpdated += InvoiceUpdatedHandler; // Подписка на событие
        serviceInvoice.AddLine(serviceItem1);
        serviceInvoice.DisplayInvoiceDetails(true);
        serviceInvoice.ServiceLocation = "Место оказания услуги";
        serviceInvoice.EstimatedHours = 5;
        serviceInvoice.DisplayInvoiceSummary();
        Console.WriteLine();

        CombinedInvoice combinedInvoice = new CombinedInvoice("CI001", DateTime.Now, true, "Клиент 3", "Комбинированный счет", "Пользователь3", DateTime.Now.AddDays(40), "Отдел финансов");
        combinedInvoice.InvoiceUpdated += InvoiceUpdatedHandler; // Подписка на событие
        combinedInvoice.AddLine(item1);
        combinedInvoice.AddLine(serviceItem1);
        combinedInvoice.DisplayInvoiceDetails(true);
        combinedInvoice.ApplyDiscount(10); // Применяем скидку
        combinedInvoice.RemoveLine("Товар A");
        combinedInvoice.DisplayInvoiceSummary();
    }

    // Обработчик события
    static void InvoiceUpdatedHandler(object sender, EventArgs e)
    {
        if (sender is Invoice invoice)
        {
            Console.WriteLine($"Счет {invoice.InvoiceNumber} был обновлен. Текущий статус: {invoice.Status}");
        }
    }
}